In [33]:
##Importing Relevant Libraries
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [42]:
##Reading the JSON file
data = json.loads(open('/content/data.json').read())

##Identifying Feature and Target for the NLP Model
words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow model/ vocabulary for tags
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X
#Iterating over all intents
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    data_X.append(pattern)
    data_y.append(intent["tag"]) ,
#adding the tag to the classes if it's not there already
    if intent["tag"] not in classes:
      classes.append(intent["tag"])
##initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))

In [43]:
#Making the data Machine-friendly
training = []
out_empty = [0] * len(classes)
##Creating the bag of words model
for idx, doc in enumerate(data_X):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)

    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    training.append([bow, output_row])
random.shuffle(training)
training = np.array(training, dtype = object)
train_X = np.array(list(training[:,0]))
train_Y = np.array(list(training[:,1]))

In [44]:
#Building the Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout (0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout (0.5))
model.add(Dense(len (train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay = 1e-6)
model. compile(loss= 'categorical_crossentropy',
        optimizer=adam,
        metrics=["accuracy"])
print (model. summary())
model. fit(x=train_X, y=train_Y, epochs=150, verbose=1)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               11904     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 15)                975       
                                                                 
Total params: 21135 (82.56 KB)
Trainable params: 21135 (82.56 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
1

In [45]:
#Preorocessing the Input
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0]*len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)



def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result





In [46]:
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res>thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
      return_list.append(labels[r[0]])
    return return_list

In [ ]:
print("Press 0 if you don't want to chat with me...")
while True:
  message = input("")
  if message == "0":
    break
  intents = pred_class(message, words, classes)
  result = get_response(intents, data)
  print(result)


Press 0 if you don't want to chat with me...
who is president?
1/1 [==============================] - 0s 81ms/step
To get info about office please go to about us section
vice president
1/1 [==============================] - 0s 31ms/step
To get info about office please go to about us section
assistant secretary?
1/1 [==============================] - 0s 24ms/step
To get info about office please go to about us section
who is assistant secretary?
1/1 [==============================] - 0s 24ms/step
To get info about office please go to about us section
what's up dude?
1/1 [==============================] - 0s 25ms/step
Good!Thank you
